- RBMT : 언어학자 -> SMT : 언어학자를 해고할수록 성능 증가(n-gram)
- seq2seq는 사실 문제가 있음 -> attention 필수(기억력의 문제 보정)
- 단어 시퀀스에 확률을 할당하기 위한 학습 방법이 이전 단어로부터 다음 단어을 예측하는 것.
- seq2seq로 만든 챗봇은 open domain
- 마지막 단어의 hidden state = context
- beam search : 성능에 영향 큼
- 숙제 : 서브클래싱 구현

## 1. BLEU

## Sentence BLEU score

파이썬 자연어 처리 패키지 NLTK는 기계가 생성한 텍스트 데이터에 대해서 성능을 평가하는 BLEU score를 제공합니다.

In [1]:
import nltk
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
import numpy as np
import re
import os
import wget
import urllib
import sentencepiece as sp
import csv
import urllib3
import zipfile
import shutil

C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\5CG7092POZ\Anaconda3\envs\tf

In [3]:
ref = [["this", "is", "a", "test"], ["this", "is" "test"]]
cand = ["this", "is", "a", "test"]
score = nltk.translate.bleu_score.sentence_bleu(ref, cand)
print(score)

1.0


## Corpus BLEU score

NLTK는 절이나 문장과 같은 다수의 문장의 묶음에 대해서 BLEU를 측정하는 nltk.translate.bleu_score.corpus_bleu()를 제공합니다. 여기서 refs는 "토큰들의 리스트의 리스트의 리스트" 삼중 리스트여야 합니다. 또한 cand는 "토큰들의 리스트의 리스트"이어야 합니다. 설명만으로는 조금 헷갈리므로 예제를 통해 이해해봅시다.

In [4]:
# two refs for one document
refs = [[["this", "is", "a", "test"], ["this", "is" "test"]]]
cands = [["this", "is", "a", "test"]]
score = nltk.translate.bleu_score.corpus_bleu(refs, cands)
print(score)

1.0


## Cumulative and Individual BLEU score

NTLK의 BLEU는 다른 n-gram들에 대해서 가중치를 달리하여 계산할 수 있도록 해줍니다.  
예를 들어 unigram에만 가중을 100% 주고, 다른 2, 3, 4-gram에 대해서 가중을 주고 싶지않다면 다음과 같이 수행할 수 있습니다.

In [5]:
# 1-gram individual BLEU
ref = [["this", "is", "small", "test"]]
cand = ["this", "is", "a", "test"]
score = nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(1, 0, 0, 0))
print(score)

0.75


C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
C:\Users\5CG7092POZ\Anaconda3\envs\tf2.3\lib\site-packages\nltk\translate\bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


위 예제를 각각의 n-gram에 대해서 수행해봅시다.

각 개별적인 n-gram 스코어로부터 각각에게 가중을 주어 가중 기하 평균을 구합니다.  
이를 BLEU-4라고 부릅니다. 이는 nltk.translate.bleu_score.sentence_bleu나 nltk.translate.bleu_score.corpus_bleu의 기본값입니다.

BLEU-1, BLEU-2, BLEU-3, BLEU-4를 각각 구하면 다음과 같습니다.

In [ ]:
# cumulative BLEU scores
ref = [["this", "is", "small", "test"]]
cand = ["this", "is", "a", "test"]
print("Cumulative 1-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(1, 0, 0, 0)))
print("Cumulative 2-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(0.5, 0.5, 0, 0)))
print("Cumulative 3-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(0.33, 0.33, 0.33, 0)))
print("Cumulative 4-gram: %f" % nltk.translate.bleu_score.sentence_bleu(ref, cand, weights=(0.25, 0.25, 0.25, 0.25)))

Cumulative 1-gram: 0.750000
Cumulative 2-gram: 0.500000
Cumulative 3-gram: 0.632878
Cumulative 4-gram: 0.707107


/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


# BLEU 구현하기

In [ ]:
from collections import Counter
import numpy as np
from nltk import ngrams

## Count : 단순 카운트 함수 구현

In [ ]:
# 단순 카운트 함수
def simple_count(tokens, n): # 토큰화 된 cand 문장, n-gram에서의 n 이 두 가지를 인자로 받음.
    return Counter(ngrams(tokens, n)) #문장에서 n-gram을 카운트

단순 카운트 구현 함수 테스트

In [ ]:
cand = "It is a guide to action which ensures that the military always obeys the commands of the party."
tokens = cand.split() #단어 토큰화
result = simple_count(tokens, 1) #토큰화 된 문장, 유니그램의 개수를 구하고자 한다면 n=1
print(result)

Counter({('the',): 3, ('It',): 1, ('is',): 1, ('a',): 1, ('guide',): 1, ('to',): 1, ('action',): 1, ('which',): 1, ('ensures',): 1, ('that',): 1, ('military',): 1, ('always',): 1, ('obeys',): 1, ('commands',): 1, ('of',): 1, ('party.',): 1})


In [ ]:
cand = "the the the the the the the"
tokens = cand.split() #단어 토큰화
result = simple_count(tokens, 1)
print(result)

Counter({('the',): 7})


## Count_clip 함수 구현

In [ ]:
def count_clip(cand, ref_list, n):
    cnt_ca = simple_count(cand, n)
    # Ca 문장에서 n-gram 카운트
    temp = dict()

    for ref in ref_list: # 다수의 Ref 문장에 대해서 이하 반복
        cnt_ref = simple_count(ref, n)
        # Ref 문장에서 n-gram 카운트

        for n_gram in cnt_ref: # 모든 Ref에 대해서 비교하여 특정 n-gram이 하나의 Ref에 가장 많이 등장한 횟수를 저장
            if n_gram in temp:
                temp[n_gram] = max(cnt_ref[n_gram], temp[n_gram]) # max_ref_count
            else:
                temp[n_gram] = cnt_ref[n_gram]

    return {
        n_gram: min(cnt_ca.get(n_gram, 0), temp.get(n_gram, 0)) for n_gram in cnt_ca
        # count_clip=min(count, max_ref_count)
        # 위의 get은 찾고자 하는 n-gram이 없으면 0을 반환한다.
     }

count_clip 함수는 cand 문장과 ref 문장들, 그리고 카운트 단위가 되는 n-gram에서의 n의 값 이 세 가지를 인자로 입력받아서 countclip을 수행합니다. 여기서는 유니그램 정밀도를 구현하고 있으므로 역시나 n=1로 하여 함수를 실행하면 됩니다.  

또한 count_clip 함수 내부에는 기존에 구현했던 simple_count 함수가 사용된 것을 확인할 수 있습니다. Countclip을 구하기 위해서는 Max_Ref_Count값과 비교하기 위해 Count값이 필요하기 때문입니다. Example 2를 통해 함수가 정상 작동되는지 확인해봅시다.

In [ ]:
cand = "the the the the the the the"
refs = [
    "the cat is on the mat",
    "there is a cat on the mat"
]
result = count_clip(cand.split(),list(map(lambda ref: ref.split(), refs)),1)
print(result)

{('the',): 2}


동일한 예제 문장에 대해서 위의 simple_count 함수는 the가 7개로 카운트되었던 것과는 달리 이번에는 2개로 카운트되었습니다. 이제 위의 두 함수를 사용하여 예제 문장에 대해서 보정된 정밀도를 연산하는 함수를 modified_precision란 이름의 함수로 구현해봅시다.

## 보정된 유니그램 정밀도 구현하기

In [ ]:
def modified_precision(cand, ref_list, n):
    clip = count_clip(cand, ref_list, n) 
    total_clip = sum(clip.values()) # 분자

    ct = simple_count(cand, n)
    total_ct = sum(ct.values()) #분모

    if total_ct==0: # n-gram의 n이 커졌을 때 분모가 0이 되는 것을 방지
      total_ct=1

    return (total_clip / total_ct) # 보정된 정밀도
    # count_clip의 합을 분자로 하고 단순 count의 합을 분모로 하면 보정된 정밀도

In [ ]:
result = modified_precision(cand.split(),list(map(lambda ref: ref.split(), refs)),1) # 유니그램이므로 n = 1
print(result)

0.2857142857142857


소수 값이 나오는데 이는 2/7의 값을 의미합니다. 

이제부터 설명에서 언급하는 "정밀도"는 기본적으로 보정된 정밀도(Modified Precision)라고 가정합니다. 정밀도를 보정하므로서 Ca에서 발생하는 단어 중복에 대한 문제점은 해결되었습니다. 하지만 유니그램 정밀도가 가지는 본질적인 문제점있기에 이제는 유니그램을 넘어 바이그램, 트라이그램 등과 같이 n-gram으로 확장해야 합니다.  

앞서 구현한 함수 simple_count, count_clip, modified_precision은 모두 n-gram의 n을 함수의 인자로 받으므로, n을 1대신 다른 값을 넣어서 실습해보면 바이그램, 트라이그램 등에 대해서도 보정된 정밀도를 구할 수 있습니다.

## 짧은 문장 길이에 대한 패널티(Brevity Penalty)

Ref가 1개라면 Ca와 Ref의 두 문장의 길이만을 가지고 계산하면 되겠지만 여기서는 Ref가 여러 개일 때를 가정하고 있으므로 r은 "모든 Ref들 중에서 Ca와 가장 길이 차이가 작은 Ref의 길이"로 합니다. r을 구하는 코드는 아래와 같습니다.

In [ ]:
def closest_ref_length(cand, ref_list): # Ca 길이와 가장 근접한 Ref의 길이를 리턴하는 함수
    ca_len = len(cand) # ca 길이
    ref_lens = (len(ref) for ref in ref_list) # Ref들의 길이
    closest_ref_len = min(ref_lens, key=lambda ref_len: (abs(ref_len - ca_len), ref_len))
    # 길이 차이를 최소화하는 Ref를 찾아서 Ref의 길이를 리턴
    return closest_ref_len

만약 Ca와 길이가 정확히 동일한 Ref가 있다면 길이 차이가 0인 최고 수준의 매치(best match length)입니다. 또한 만약 서로 다른 길이의 Ref이지만 Ca와 길이 차이가 동일한 경우에는 더 작은 길이의 Ref를 택합니다. 예를 들어 Ca가 길이가 10인데, Ref 1, 2가 각각 9와 11이라면 길이 차이는 동일하게 1밖에 나지 않지만 9를 택합니다. closest_ref_length 함수를 통해 r을 구했다면, 이제 BP를 구하는 함수 brevity_penalty를 구현해봅시다.

In [ ]:
def brevity_penalty(cand, ref_list):
    ca_len = len(cand)
    ref_len = closest_ref_length(cand, ref_list)

    if ca_len > ref_len:
        return 1
    elif ca_len == 0 :
    # cand가 비어있다면 BP = 0 → BLEU = 0.0
        return 0
    else:
        return np.exp(1 - ref_len/ca_len)

## BLEU 함수 구현하기

이제 최종적으로 BLEU 점수를 계산하는 함수 bleu_score를 구현해봅시다.

In [ ]:
def bleu_score(cand, ref_list, weights=[0.25, 0.25, 0.25, 0.25]):
    bp = brevity_penalty(cand, ref_list) # 브레버티 패널티, BP

    p_n = [modified_precision(cand, ref_list, n=n) for n, _ in enumerate(weights,start=1)] 
    #p1, p2, p3, ..., pn
    score = np.sum([w_i * np.log(p_i) if p_i != 0 else 0 for w_i, p_i in zip(weights, p_n)])
    return bp * np.exp(score)

위 함수가 동작하기 위해서는 앞서 구현한 simple_count, count_clip, modified_precision, brevity_penalty 4개의 함수 또한 모두 구현되어져 있어야 합니다. 지금까지 구현한 BLEU 코드로 계산된 점수와 NLTK 패키지에 이미 구현되어져 있는 BLEU 코드로 계산된 점수를 비교해봅시다.

## NLTK의 BLEU Vs. 구현한 BLEU 함수

In [ ]:
import nltk.translate.bleu_score as bleu


cand = "It is a guide to action which ensures that the military always obeys the commands of the party"
refs = [
    "It is a guide to action that ensures that the military will forever heed Party commands",
    "It is the guiding principle which guarantees the military forces always being under the command of the Party",
    "It is the practical guide for the army always to heed the directions of the party"
]

# 이번 챕터에서 구현한 코드로 계산한 BLEU 점수
print(bleu_score(cand.split(),list(map(lambda ref: ref.split(), refs))))
# NLTK 패키지 구현되어져 있는 코드로 계산한 BLEU 점수
print(bleu.nltk.translate.bleu_score.sentence_bleu(list(map(lambda ref: ref.split(), refs)),cand.split()))

0.5045666840058485
0.5045666840058485
